# UDPipe 2



## Setup environment

In [ ]:
! nvidia-smi

In [ ]:
! wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64
! dpkg -i /content/cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64
! sudo apt-get update
! sudo apt-get install cuda-10.0.*
! apt autoremove

# You will need to manually download these files (https://gist.github.com/matheustguimaraes/43e0b65aa534db4df2918f835b9b361d?permalink_comment_id=3433448)
! dpkg -i "/content/drive/MyDrive/Studies/Mestrado/Experimentos Tagging Porttinari-base/libs/libcudnn7_7.4.2.24-1+cuda10.0_amd64.deb"
! dpkg -i "/content/drive/MyDrive/Studies/Mestrado/Experimentos Tagging Porttinari-base/libs/libcudnn7-dev_7.4.2.24-1+cuda10.0_amd64.deb"
! dpkg -i "/content/drive/MyDrive/Studies/Mestrado/Experimentos Tagging Porttinari-base/libs/libcudnn7-doc_7.4.2.24-1+cuda10.0_amd64.deb"
! ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.7*

Restart the environment

In [ ]:
! rm -rf /content/UD_Portuguese-Porttinari

In [ ]:
# Porttinari
! git clone https://github.com/huberemanuel/UD_Portuguese-Porttinari.git

## Setting up UDPipe 2

In [ ]:
!rm -rf udpipe2

In [ ]:
%%bash
git clone https://github.com/ufal/udpipe udpipe2

In [ ]:
%%bash
cd udpipe2 
git checkout udpipe-2

### Installing dependencies

In [ ]:
! pip uninstall tensorflow tensorflow-gpu -y
! pip install tensorflow-gpu==1.15.4 

In [ ]:
! nvidia-smi

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

Install another udpipe dependency

In [ ]:
%%bash
cd udpipe2
git clone https://github.com/ufal/chu_liu_edmonds.git

In [ ]:
%%bash
pip install -e udpipe2/chu_liu_edmonds/

Installing wembedding_service dependency

In [ ]:
%%bash
cd udpipe2
rm -rf wembedding_service
git clone https://github.com/ufal/wembedding_service

## Training udpipe

In [ ]:
%%bash
rm -rf udpipe2/my-model

Training from zero:

In [ ]:
TRAIN="/content/UD_Portuguese-Porttinari/pt_porttinari-ud-train.conllu"
DEV="/content/UD_Portuguese-Porttinari/pt_porttinari-ud-dev.conllu"
TEST="/content/UD_Portuguese-Porttinari/pt_porttinari-ud-test.conllu"
! mkdir -p /content/udpipe2/my-model/artifacts
! cp {TRAIN} /content/udpipe2/my-model/artifacts/train.conllu
! cp {DEV} /content/udpipe2/my-model/artifacts/dev.conllu
! cp {TEST} /content/udpipe2/my-model/artifacts/test.conllu

In [ ]:
for i in range(6):
    run_id = 46 + i
    ! rm -rf udpipe2/my-model/
    ! cd udpipe2 && python udpipe2.py my-model --train {TRAIN} \
                                               --parse 0 \
                                               --dev {DEV} \
                                               --test {TEST} \
                                               --epochs 8:1e-3,8:1e-4 \
                                               --batch_size 128 \
                                               --tags "UPOS" \
                                               --wembedding_model "neuralmind/bert-large-portuguese-cased" \
                                               --seed {run_id}

    ! cd udpipe2 && python udpipe2.py my-model --predict --predict_input {TRAIN} --predict_output train.conllu
    ! cd udpipe2 && python udpipe2_eval.py {TRAIN} train.conllu --verbose > udpipe2/my-model/train_result.txt
    ! cd udpipe2 && python udpipe2.py my-model --predict --predict_input {TEST} --predict_output test.conllu
    ! cd udpipe2 && python udpipe2_eval.py {TEST} test.conllu --verbose > udpipe2/my-model/test_result.txt

    ! mkdir -p runs/{run_id}
    ! cp udpipe2/*.conllu udpipe2/my-model/
    ! zip -r -9 runs/logs_{run_id}.zip udpipe2/my-model/
    # Comment or put your specific location to store the experiments :)
    ! mv runs/logs_{run_id}.zip "/content/drive/MyDrive/Studies/Mestrado/Experimentos Tagging Porttinari-base/udpipe2/"